In [ ]:
import graphtik
import math
import operator
import osd
import nec_tables as nec

In [ ]:
def get_wire_size(current, mat, temp):
    table = nec.cable_ampacity_310_16[mat][temp]
    wire_size = osd.lookup(current, table, keys=False)
    return (wire_size, nec.cable_ampacity_310_16[mat][temp][wire_size])

In [ ]:
def compare_wire_sizes(cond_a, cond_b):
    """
    arguments are tuples (wire_size, ampacity)
    """
    if cond_a[1] > cond_b[1]:
        return cond_a
    else:
        return cond_b

In [ ]:
graph = graphtik.compose('base_wire_size',
                         graphtik.operation(name='ccc_derate', needs=['ccc_count', 'table'], provides=['ccc_derate'])(osd.lookup),
                         graphtik.operation(name='amb_temp_derate', needs=['amb_temp', 'wire_insulation_temp'], provides=['amb_derate'])(osd.get_ambient_temp_derate),
                         graphtik.operation(name='cond_of_use_derates', needs=['ccc_derate', 'amb_derate'], provides=['total_derate'])(operator.mul),
                         graphtik.operation(name='current_per_conductor', needs=['current', 'parallel_conductors'], provides=['parallel_current'])(operator.mul),
                         graphtik.operation(name='current_terminal_check', needs=['parallel_current', 'ocpd_derate'], provides=['terminal_check_current'])(operator.truediv),
                         graphtik.operation(name='cond_use_current', needs=['total_derate', 'parallel_current'], provides=['cond_use_check_current'])(operator.mul),
                         graphtik.operation(name='wire_size_terminal', needs=['terminal_check_current', 'wire_material', 'terminal_temp_rating'], provides=['terminal_min_wire'])(get_wire_size),
                         graphtik.operation(name='wire_size_cond_use', needs=['cond_use_check_current', 'wire_material', 'wire_insulation_temp'], provides=['cond_use_min_wire'])(get_wire_size),
                         graphtik.operation(name='compare_wire_sizes', needs=['terminal_min_wire', 'cond_use_min_wire'], provides=['min_wire_size'])(compare_wire_sizes)
                        )

In [ ]:
results = graph(**{'ccc_count': 4, 'table': nec.ccc_count_derate,
                     'amb_temp': 32, 'wire_insulation_temp': 90,
                     'current': 60, 'parallel_conductors': 1,
                     'ocpd_derate': 0.8,
                     'wire_material': 'Cu', 'terminal_temp_rating': 75})

In [ ]:
for key, val in results.items():
    print('{}: {}'.format(key, val))

In [ ]:
graph.plot(filename='base_wire_size.pdf')